<a href="https://colab.research.google.com/github/swq2101/i-dub-thee/blob/main/i_dub_thee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir "/content/sample_data/samples"
!unzip "/content/drive/MyDrive/i-dub-thee/samples.zip" -d "/content/sample_data/samples"

In [6]:
import pandas as pd
import os

In [21]:
#load files and get classes
subdirectories = []
classes = {"relative_path":[], "class_id":[]}

root = "/content/sample_data/samples/samples"
with os.scandir(root) as entries:
  for entry in entries:
        if entry.name != ".DS_Store":
          subdirectories.append(entry.name)

class_id = 0

for sd in subdirectories:
  with os.scandir(root + "/" + sd) as entries:
    for entry in entries:
      if entry.name != ".DS_Store":
          classes["relative_path"].append(root + "/" + sd + "/" + entry.name)
          classes["class_id"].append(class_id)
    class_id += 1

df = pd.DataFrame(data=classes)



,relative_path,class_id
0,/content/sample_data/samples/samples/yuriLowen...,0
1,/content/sample_data/samples/samples/yuriLowen...,0
2,/content/sample_data/samples/samples/yuriLowen...,0
3,/content/sample_data/samples/samples/yuriLowen...,0
4,/content/sample_data/samples/samples/yuriLowen...,0
...,...,...
142,/content/sample_data/samples/samples/matthewMe...,2
143,/content/sample_data/samples/samples/matthewMe...,2
144,/content/sample_data/samples/samples/matthewMe...,2
145,/content/sample_data/samples/samples/matthewMe...,2
